# Install required package and download the EnergyScope model

In [1]:
import os
import pandas as pd
import bw2data as bd
from mescal import *
from energyscope.models import Model
from energyscope.energyscope import Energyscope
from energyscope.result import postprocessing

## Initialize the EnergyScope model

In [ ]:
# AMPL licence 
path_to_ampl_licence = r'C:\Users\matth\ampl' # Path to the AMPL licence file
os.environ['PATH'] = path_to_ampl_licence+':'+os.environ['PATH']

In [ ]:
# Define the solver options
solver_options = {
    'solver': 'gurobi',
    'solver_msg': 0,
}

In [ ]:
# Initialize the QC model with .mod and .dat files
model = Model(
    mod_files=[
        '..//01_Model//07_Quebec_v1//QC_es_main.mod',
    ],
    dat_files=[
        '..//01_Model//07_Quebec_v1//QC_data.dat',
        '..//01_Model//07_Quebec_v1//QC_mob_techs_dist_B2D.dat',
        '..//01_Model//07_Quebec_v1//QC_techs_B2D.dat',
        '..//01_Model//07_Quebec_v1//QC_mob_params.dat',
        '..//01_Model//07_Quebec_v1//QC_scenarios.dat',
    ],
)

In [ ]:
# Initialize the EnergyScope model
es = Energyscope(model=model, solver_options=solver_options)

In [ ]:
# Solve the model and get results
results = postprocessing(es.calc())

# Generate of PB indicators 

In [4]:
path_inputs = '../01_Notebooks/Data/'
path_model = '../02_Model/'
path_results = '../03_Results/'

## Load data and initialize the ESM class

In [5]:
# Load the data
mapping = pd.read_csv(path_inputs+'mapping_3.9.1_linked.csv')
unit_conversion = pd.read_csv(path_inputs+'unit_conversion_3.9.1.csv')
techno_compositions = pd.read_csv(path_inputs+'technology_compositions.csv')
tech_specifics = pd.read_csv(path_inputs+'technology_specifics.csv')
efficiency = pd.read_csv(path_inputs+'efficiency.csv')
lifetime = pd.read_csv(path_inputs+'lifetime.csv')
mapping_es_flows_to_cpc = pd.read_csv(path_inputs+'mapping_esm_flows_to_CPC.csv')
impact_abbrev = pd.read_csv(path_inputs+'impact_abbrev.csv')

In [6]:
# Load the model from energyscope model
model = results.parameters['layers_in_out'].reset_index().rename(columns={'index0':'Name', 'index1':'Flow', 'layers_in_out':'Amount'}).drop(columns=['Run'])
model = model[model['Amount'] != 0]

Gurobi 9.1.1: optimal solution; objective 8151.373244
4859 simplex iterations
1 branch-and-cut nodes
plus 6691 simplex iterations for intbasis

"option abs_boundtol 1.2244962846443698e-15;"
will change deduced dual values.



In [8]:
# Databases names 
name_main_database = 'ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized'
name_spatialized_biosphere_db = 'biosphere3_spatialized_flows'
name_es_database = 'EnergyScope_CA-QC_2050'

In [9]:
# Set up your Brightway project
bd.projects.set_current('ecoinvent3.9.1A')

In [10]:
main_db = Database(name_main_database)

Getting activity data


100%|██████████| 38801/38801 [00:00<00:00, 79342.45it/s] 


Adding exchange data to activities


100%|██████████| 1125167/1125167 [01:17<00:00, 14472.84it/s]


Filling out exchange data


100%|██████████| 38801/38801 [00:08<00:00, 4690.97it/s] 


Loaded ecoinvent_cutoff_3.9.1_image_SSP2-Base_2050+truck_carculator regionalized for EF from brightway!


In [11]:
ranking_best_ecoinvent_locations_for_QC = [
    'CA-QC', # Quebec
    'CA', # Canada
    'CAN', # Canada in IMAGE
    'CAZ', # Canada - Australia - New Zealand in REMIND
    'RNA', # North America
    'US', # United States
    'USA', # United States in REMIND and IMAGE
    'GLO', # Global average 
    'RoW', # Rest of the world
]

In [12]:
# Add CPC categories to the main database
main_db.add_CPC_categories()

In [13]:
# Change the main database name if needed
if mapping['Database'].iloc[0] != name_main_database:
    mapping['Database'] = len(mapping) * [name_main_database]
missing_flows = main_db.test_mapping_file(mapping)

Mapping successfully linked to the database


In [14]:
esm = ESM(
    # Mandatory inputs
    mapping=mapping,
    unit_conversion=unit_conversion,
    model=model,
    mapping_esm_flows_to_CPC_cat=mapping_es_flows_to_cpc,
    main_database=main_db,
    esm_db_name=name_es_database,
    
    # Optional inputs
    technology_compositions=techno_compositions,
    tech_specifics=tech_specifics,
    lifetime=lifetime,
    efficiency=efficiency,
    regionalize_foregrounds=True,
    accepted_locations=['CA-QC', 'CA'],
    locations_ranking=ranking_best_ecoinvent_locations_for_QC,
    esm_location='CA-QC',
    results_path_file=path_results,
    
    # If we want regionalized results 
    spatialized_biosphere_db=Database(name_spatialized_biosphere_db),
)

Getting activity data


100%|██████████| 30207/30207 [00:01<00:00, 27211.26it/s] 


Adding exchange data to activities


0it [00:00, ?it/s]


Filling out exchange data


100%|██████████| 30207/30207 [00:00<00:00, 2872825.29it/s]

Loaded biosphere3_spatialized_flows_EF from brightway!


In [15]:
esm.check_inputs()

List of technologies or resources that are in the model file but not in the mapping file. Their impact scores will be set to the default value.

--> ['CARBON_CAPTURE', 'CO2_CS', 'CO2_E', 'DEC_RENOVATION', 'DHN_RENOVATION', 'DIESEL_S', 'ELEC_S', 'GASOLINE_S', 'H2_S', 'HT_LT', 'LT_DEC_WH', 'LT_DHN_WH', 'NG_S', 'PLANE', 'RES_GEO', 'RES_HYDRO', 'RES_SOLAR', 'RES_WIND', 'STO_CO2', 'STO_DIE', 'STO_ELEC', 'STO_GASO', 'STO_H2', 'STO_NG']

List of technologies or resources that are in the mapping file but not in the model file (this will not be a problem in the workflow).

--> ['BATTERY', 'CAR_BEV_LOWRANGE', 'CAR_DME_D10_LOCAL', 'CAR_DME_D10_LONGD', 'DEC_TH_STORAGE', 'DHN_TH_STORAGE', 'EHP_H2_GRID', 'EHP_NG_GRID', 'EHV_GRID', 'HP_H2_GRID', 'HP_NG_GRID', 'HV_GRID', 'LP_H2_GRID', 'LP_NG_GRID', 'LV_GRID', 'MP_H2_GRID', 'MP_NG_GRID', 'MV_GRID']

List of technologies that are in the tech_specifics file but not in the mapping file (this will not be a problem in the workflow).

--> ['MOB_AVIATION', 'M

### Generate ESM database

In [17]:
# Foreground regionalization, double-counting removal, and efficiency harmonization
esm.create_esm_database()

### Starting to add construction and resource activities database ###
No location found in your ranking for iridium - platinum group metal, extraction and refinery operations
--> Have to keep the initial location: ZA
No location found in your ranking for natural gas, liquefied - natural gas production, liquefied
--> Have to keep the initial location: QA
### Construction and resource activities added to the database ###
--> Time: 21.0 seconds
### Starting to remove double-counted flows ###
### Double-counting removal done ###
--> Time: 38.0 seconds
### Starting to correct efficiency differences ###
No flow found for type(s) ['NG_HP'] in BUS_FC_HYBRID_CH4. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in CAR_FC_CH4_LOCAL. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in CAR_FC_CH4_LONGD. The efficiency of this technology cannot be adjusted.
No flow found for type(s) ['NG_HP'] in COACH_FC_HYBRID_CH4. T

Writing activities to SQLite3 database:
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:01


Title: Writing activities to SQLite3 database:
  Started: 11/22/2024 17:03:40
  Finished: 11/22/2024 17:03:41
  Total time elapsed: 00:00:01
  CPU %: 44.80
  Memory %: 11.09
### Database written ###
--> Time: 552.0 seconds


In [19]:
# Save the mapping file with new codes for later use
esm.mapping.to_csv(path_results+'mapping_with_new_codes.csv', index=False)

## Generate PB metrics 

In [20]:
# To skip the ESM database creation
# esm.mapping = pd.read_csv(path_results+'mapping_with_new_codes.csv')

In [21]:
methods = ['IMPACT World+ Midpoint 2.0.1_regionalized', 'IMPACT World+ Damage 2.0.1_regionalized']

In [22]:
# Lifetime harmonization
R_long = esm.compute_impact_scores(methods=methods)

Getting activity data


100%|██████████| 655/655 [00:00<00:00, 135721.23it/s]


Adding exchange data to activities


100%|██████████| 12627/12627 [00:00<00:00, 17243.67it/s]


Filling out exchange data


100%|██████████| 655/655 [00:01<00:00, 389.47it/s]


Loaded EnergyScope_CH_2050_EF from brightway!


In [23]:
R_long.Value *= 1e6 # from FU / kW(h)-pkm-tkm to FU / GW(h)-Mpkm-Mtkm

In [24]:
R_long.to_csv(f'{path_results}impact_scores.csv', index=False)

## Create the .mod and .dat files

In [25]:
# To skip the impact assessment step
# R_long = pd.read_csv(f'{path_results}impact_scores.csv')

In [26]:
method = 'IMPACT World+ Damage 2.0.1_regionalized'

In [27]:
metadata = {
    'ecoinvent_version': '3.9.1',
    'year': '2050',
    'iam': 'image',
    'ssp_rcp': 'SSP2-Base',
}

In [28]:
# Create .dat file
esm.normalize_lca_metrics(
    R=R_long,
    mip_gap=1e-6,
    lcia_methods=[method],
    specific_lcia_abbrev=['TTHH'],
    impact_abbrev=impact_abbrev,
    path=path_model+f'{method}/',
    metadata=metadata,
)

In [29]:
# Create the .mod file
esm.generate_mod_file_ampl(
    lcia_methods=[method],
    impact_abbrev=impact_abbrev,
    specific_lcia_abbrev=['TTHH'],
    path=path_model+f'{method}/',
    metadata=metadata,
)